In [ ]:
# Let's start with some lovely imports that should have been installed if not available by default
import json
import requests
import os
from zenroom import zenroom


In [ ]:
# Test zenroom is correctly installed and running witht the following function
def generate_random_challenge():
    """
        This function calls zenroom to generate
        a random string to be used as challenge
    """
    contract = """
        rule check version 1.0.0
        Given nothing
        When I create the random object of '512' bits
        and I rename the 'random_object' to 'challenge'
        Then print 'challenge'
    """

    try:
        result = zenroom.zencode_exec(contract, conf='debug=0')
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

    res_json = json.loads(result.output)

    print(f"Generated challenge: {res_json['challenge']}")

    return


In [ ]:
# This should print something like
# Generated challenge: MMQ1JSrCA7L4QNftLyaaSRunT4Z9+Rr2QkE+a+DWLEljtg6EroLbCj5VjLH+xba9Rv1D+3ncQHw5s/lH41IFJw==
generate_random_challenge()

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls
file = '.cred_user1.json'
if os.path.isfile(file):
    with open(file,'r') as f:
        user1_data = json.loads(f.read())
    print(f"Credentials file available for {user1_data['name']}")
else:
    user1_data = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "User One",
      "username": "user1_username",
      "email": "user1@example.org",
      "note": "me.user1.org"
    }
    with open(file,'w') as f:
        json.dump(user1_data, f)

file = 'loc_user1.json'
if os.path.isfile(file):
    with open(file,'r') as f:
        loc1_data = json.loads(f.read())
    print(f"Location file available for {loc1_data['name']}")
else:
    loc1_data = {
        "name": "OLVG",
        "lat": 52.35871773455108,
        "long": 4.916762398221842,
        "addr": "Oosterpark 9, 1091 AC Amsterdam",
        "note": "location.user1.org"
    }
    with open(file,'w') as f:
        json.dump(loc1_data, f)

file = '.cred_user2.json'
if os.path.isfile(file):
    with open(file,'r') as f:
        user2_data = json.loads(f.read())
    print(f"Credentials file available for {user2_data['name']}")
else:
    user2_data = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "User Two",
        "username": "user2",
        "email": "user2@example.org",
        "note" : "me.user2.org"
    }
    with open(file,'w') as f:
        json.dump(user2_data, f)

file = 'loc_user2.json'
if os.path.isfile(file):
    with open(file,'r') as f:
        loc2_data = json.loads(f.read())
    print(f"Location file available for {loc2_data['name']}")
else:
    loc2_data = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven",
        "note": "location.user2.org"
    }
    with open(file,'w') as f:
        json.dump(loc2_data, f)

file = 'unit_data.json'
if os.path.isfile(file):
    with open(file,'r') as f:
        unit_data = json.loads(f.read())
    print(f"Unit file available")
else:
    unit_data = {}
#     with open(file,'w') as f:
#         json.dump(unit_data, f)


file = 'res_spec_data.json'
if os.path.isfile(file):
    with open(file,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}



In [ ]:
# What endpoint are we talking to?
endpoint = 'http://65.109.11.42:9000/api'

In [ ]:
# this function is equivalent to JSON.stringify in javascript, i.e. it does not add spaces
# Although it does not seem to matter as Zenroom removes spaces
def stringify(json_obj):
    return json.dumps(json_obj, separators=(',',':'))

In [ ]:
# Get the seed from the server to later generate the keypair
def get_HMAC(email):

    variables = {
        "firstRegistration": True,
        "userData": "{\"email\": \"" + email + "\"}"
    };

    
    payload = {
      "query": """mutation ($firstRegistration: Boolean!, $userData: String!){
  
        keypairoomServer(firstRegistration: $firstRegistration, userData: $userData)
      
      }""",
      "variables": variables
    }
    

    r = requests.post(endpoint, json=payload)
    
    result = r.json()

    print(json.dumps(result, indent=2))
    
    return result

In [ ]:
def read_HMAC(file, user_data):
    # check we already have a credentials file with a HMAC
    if 'seedServerSideShard.HMAC' in user_data:
        print(f"Server HMAC available for {user_data['name']}")
        return
    if os.path.isfile(file):
        with open(file,'r') as f:
                local_data = json.loads(f.read())

    else:
        local_data = {}
        
    if 'seedServerSideShard.HMAC' not in local_data:
        res = get_HMAC(user_data['email'])
        # save the HMAC in the user data
        user_data['seedServerSideShard.HMAC'] = res['data']['keypairoomServer']
        # save data with HMAC
        with open(file,'w') as f:
            json.dump(user_data, f)
    else:
        user_data['seedServerSideShard.HMAC'] = local_data['seedServerSideShard.HMAC']
        # no need to save since we read it from file
    

In [ ]:
# Read HMAC or get it from the server
read_HMAC('.cred_user1.json', user1_data)
read_HMAC('.cred_user2.json', user2_data)

In [ ]:
# Generate the user keypair (and the mnemonic seed)
def generate_keypair(userdata):
    """
        This function calls zenroom to generate
        a keypair using the server-provided HMAC
    """
    contract = """
        Scenario 'ecdh': Create the key
        Scenario 'ethereum': Create key
        Scenario 'reflow': Create the key
        Scenario 'schnorr': Create the key
        Scenario 'eddsa': Create the key
        Scenario 'qp': Create the key


        # Loading the user name from data
        Given my name is in a 'string' named 'username'

        # Loading the answers from 3 secret questions. The user will have to pick the 3 challenges from a list 
        # and have to remember the questions - the order is not important cause Zenroom will sort alphabetically 
        # the data in input
        #
        # NOTE: the challenges will never be communicated to the server or to anybody else!
        Given I have a 'string dictionary' named 'userChallenges'

        # Loading the individual challenges, in order to have them hashed 
        # and the hashes OPTIONALLY stored by the server, to improve regeneration of the keypair
        Given I have a 'string' named 'whereParentsMet' in 'userChallenges'
        Given I have a 'string' named 'nameFirstPet' in 'userChallenges'
        Given I have a 'string' named 'whereHomeTown' in 'userChallenges'
        Given I have a 'string' named 'nameFirstTeacher' in 'userChallenges'
        Given I have a 'string' named 'nameMotherMaid' in 'userChallenges'

        # Loading the pbkdf received from the server, containing a signed hash of known data
        Given that I have a 'base64' named 'seedServerSideShard.HMAC' 

        # Save the backup for mnemonic dump, before factoring with the salt
        # it is shortened to 16 bytes by hashing sha512 the KDF and taking the first 16 bytes
        When I create the key derivation of 'userChallenges'
        and I create the hash of 'key derivation' using 'sha512'
        and I split the leftmost '16' bytes of 'hash'
        and I delete the 'key derivation'
        and I delete the 'hash'
        and I rename the 'leftmost' to 'seed'

        # Hash again the user's challenges with salt for the seed root
        When I rename 'seedServerSideShard.HMAC' to 'salt'
        and I create the key derivation of 'seed' with password 'salt'
        and I rename the 'key derivation' to 'seed.root'

        # In the following flow the order should NOT be changed

        When I create the hash of 'seed.root'
        When I rename the 'hash' to 'seed.ecdh'

        When I create the hash of 'seed.ecdh'
        When I rename the 'hash' to 'seed.eddsa'

        When I create the hash of 'seed.eddsa'
        When I rename the 'hash' to 'seed.ethereum'

        When I create the hash of 'seed.ethereum'
        When I rename the 'hash' to 'seed.reflow'

        When I create the hash of 'seed.reflow'
        When I rename the 'hash' to 'seed.schnorr'

        # end of the sorted creation flow

        When I create the ecdh key with secret key 'seed.ecdh'
        When I create the eddsa key with secret key 'seed.eddsa'
        When I create the ethereum key with secret key 'seed.ethereum'
        When I create the reflow key with secret key 'seed.reflow'
        When I create the schnorr key with secret key 'seed.schnorr'

        When I create the ecdh public key
        When I create the eddsa public key
        When I create the ethereum address
        When I create the reflow public key
        When I create the schnorr public key

        # Creating the hashes of the single challenges, to OPTIONALLY help 
        # regeneration of the keypair

        When I create the 'base64 dictionary'
        and I rename the 'base64 dictionary' to 'hashedAnswers'

        When I create the key derivation of 'whereParentsMet'
        and I rename the 'key derivation' to 'whereParentsMet.kdf'
        When I insert 'whereParentsMet.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameFirstPet'
        and I rename the 'key derivation' to 'nameFirstPet.kdf'
        When I insert 'nameFirstPet.kdf' in 'hashedAnswers'

        When I create the key derivation of 'whereHomeTown'
        and I rename the 'key derivation' to 'whereHomeTown.kdf'
        When I insert 'whereHomeTown.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameFirstTeacher'
        and I rename the 'key derivation' to 'nameFirstTeacher.kdf'
        When I insert 'nameFirstTeacher.kdf' in 'hashedAnswers'

        When I create the key derivation of 'nameMotherMaid'
        and I rename the 'key derivation' to 'nameMotherMaid.kdf'
        When I insert 'nameMotherMaid.kdf' in 'hashedAnswers'


        # This prints the keyring
        Then print the 'keyring' 

        # this prints the hashes of the challenges
        # Then print the 'hashedAnswers'

        # This prints the seed for the private keys as mnemonic 
        Then print the 'seed' as 'mnemonic'

        Then print the 'ecdh public key'
        Then print the 'eddsa public key'
        Then print the 'ethereum address'
        Then print the 'reflow public key'
        Then print the 'schnorr public key'
    """
    
    data = json.dumps(userdata)

    try:
        result = zenroom.zencode_exec(contract, data=data)
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

#     print(f'result: {result}')
    res_json = json.loads(result.output)

    print(f"Generated keypair data: {json.dumps(res_json, indent=2)}")

    return res_json


In [ ]:
def read_keypair(file, user_data):
    if ('seed' in user_data and 'eddsa_public_key' in user_data and \
            'keyring' in user_data and 'eddsa' in user_data['keyring']):
        print(f"Keypair available for {user_data['name']}")
        return
        
    if os.path.isfile(file):
        with open(file,'r') as f:
                tmp_user_data = json.loads(f.read())
    else:
        tmp_user_data = {}

    if not ('seed' in tmp_user_data and 'eddsa_public_key' in tmp_user_data and \
            'keyring' in tmp_user_data and 'eddsa' in tmp_user_data['keyring']):

        res = generate_keypair(user_data)
        # Update data in user data
        user_data['seed'] = res['seed']
        user_data['eddsa_public_key'] = res['eddsa_public_key']
        user_data['keyring'] = {}
        user_data['keyring']['eddsa'] = res['keyring']['eddsa']

        with open(file,'w') as f:
            json.dump(user_data, f)
    else:
        print(f"Keypair available from file for {user_data['name']}")
        user_data['seed'] = tmp_user_data['seed']
        user_data['eddsa_public_key'] = tmp_user_data['eddsa_public_key']
        user_data['keyring'] = {}
        user_data['keyring']['eddsa'] = tmp_user_data['keyring']['eddsa']



In [ ]:
# Read the keypair
read_keypair('.cred_user1.json', user1_data)
read_keypair('.cred_user2.json', user2_data)

In [ ]:
# Create the person using their public key

def create_Person(name, username, email, eddsaPublicKey):

    variables = {
    "person": {
        "name": name,
        "user": username,
        "email": email,
        "eddsaPublicKey": eddsaPublicKey
        }
    }

    payload = {
      "query": """mutation ($person: PersonCreateParams!){
        createPerson(person: $person)
        {
            agent{
                id
                name
                user
                email
                eddsaPublicKey
            }
        }
       }""",
      "variables": json.dumps(variables)
    }

#     print(json.dumps(payload, indent=2))

    # Temporarily: we need a key to create a person, before email authentication is implemented
    file = '.credentials.json'
    assert os.path.isfile(file)

    with open(file) as f:
        data = json.load(f)
        SECRET_KEY = data['key']
        
    r = requests.post(endpoint, json=payload, headers={'zenflows-admin': SECRET_KEY})
    
    result = r.json()

    print(json.dumps(result, indent=2))
    
    return result

In [ ]:
def get_id_person(file, user_data):
    if 'id' in user_data:
        print(f"Id available for {user_data['name']}")
        return
    
    if os.path.isfile(file):
        with open(file,'r') as f:
                tmp_user_data = json.loads(f.read())
    else:
        tmp_user_data = {}

    if not 'id' in tmp_user_data:
        res = create_Person(user_data['name'], user_data['username'], user_data['email'], user_data['eddsa_public_key'])
        user_data['id'] = res['data']['createPerson']['agent']['id']
#         print(json.dumps(user_data, indent=2))
        with open(file,'w') as f:
            json.dump(user_data, f)
    else:
        print(f"Id available from file for {user_data['name']}")
        user_data['id'] = tmp_user_data['id']
    

In [ ]:
# read or get id of the person
get_id_person('.cred_user1.json', user1_data)
get_id_person('.cred_user2.json', user2_data)

In [ ]:
# sign and send each request now that we have a registered public key
import base64

def send_signed(query, variables, username, eddsa):

    sign_script = """
    Scenario eddsa: sign a graph query
    Given I have a 'base64' named 'gql'
    Given I have a 'keyring'

    # Fix Apollo's mingling with query string
    When I remove spaces in 'gql'
    and I compact ascii strings in 'gql'

    When I create the eddsa signature of 'gql'
    And I create the hash of 'gql'

    Then print 'eddsa signature' as 'base64'
    Then print 'gql' as 'base64'
    Then print 'hash' as 'hex'
    """
    
    body = f'{{"query": "{query}", "variables": {stringify(variables)}}}'

#     print("Body")
#     print(body)

    zenKeys = stringify({
        "keyring": {
            "eddsa": eddsa
        }
    })

    encoded_body = base64.b64encode(body.encode('utf8')).decode('ascii')

    zenData = f"""
    {{
        "gql": "{encoded_body}"
    }}"""

#     print("zenData")
#     print(zenData)
    
    try:
        result = zenroom.zencode_exec(sign_script, keys=zenKeys, data=zenData)
    except Exception as e:
        print(f'Exception in zenroom call: {e}')
        return None

    # print("Zenroom result")
    # print(result)
    
    res_json = json.loads(result.output)

#     print("Generated signature")
#     print(json.dumps(res_json, indent=2))
    
    # Reset the headears
    headers = {}
    headers['content-type'] = 'application/json'

    headers['zenflows-sign'] = res_json['eddsa_signature']   
    headers['zenflows-user'] = username
    headers['zenflows-hash'] = res_json['hash']

    payload = {
        "query": query,
        "variables": variables
    }
    
    r = requests.post(endpoint, json=payload, headers=headers)

    result = r.json()

#     print("Response")
#     print(json.dumps(result, indent=2))
    
    return result

In [ ]:
def get_location_id(file, user_data, loc_data):

    if 'id' in loc_data:
        print(f"Location id available for {loc_data['name']}")
        return

    # check we already have a location file with an id
    if os.path.isfile(file):
        with open(file,'r') as f:
                temp_loc_data = json.loads(f.read())
    else:
        temp_loc_data = {}


    if 'id' not in temp_loc_data:
        # Register location
        # Produce the query and variables vars to be signed
        variables = {
            "location": {
                "name": loc_data['name'],
                "alt": 0,
                "lat": loc_data['lat'],
                "long": loc_data['long'],
                "mappableAddress": loc_data['addr'],
                "note": loc_data['note']
            }
        }

        query = """mutation($location: SpatialThingCreateParams!) {
                createSpatialThing(spatialThing: $location) {
                    spatialThing {
                        id
                    }
                }
            }"""

        res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'])

        # save the id in the location data
        loc_data['id'] = res['data']['createSpatialThing']['spatialThing']['id']

        # save data with id
        with open(file,'w') as f:
            json.dump(loc_data, f)
    else:
        print(f"Location id available in file for {loc_data['name']}")
        loc_data['id'] = temp_loc_data['id']

In [ ]:
# Read of get the location id
get_location_id('loc_user1.json', user1_data, loc1_data)
get_location_id('loc_user2.json', user2_data, loc2_data)

In [ ]:
def get_unit_id(file, user_data, unit_data, name, label, symbol):
    
    if name in unit_data:
        print(f"Unit {name} available")
        return

    # check we already have a unit file with an id
    if os.path.isfile(file):
        with open(file,'r') as f:
                temp_unit_data = json.loads(f.read())
    else:
        temp_unit_data = {}

    if name not in temp_unit_data:

        # Produce the query and variables vars to be signed
        variables = {
                    "unit": {
                            "label": label,
                            "symbol": symbol
                            }
                }


        query = """mutation($unit:UnitCreateParams!) {
                createUnit(unit: $unit) {
                    unit {
                        id
                    }
                }
              }"""

        res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'])
#         print(res)

        # save the unit info
        unit_data[f'{name}'] = {}
        unit_data[f'{name}']['label'] = label
        unit_data[f'{name}']['symbol'] = symbol
        unit_data[f'{name}']['id'] = res['data']['createUnit']['unit']['id']

        # save data with id
        with open(file,'w') as f:
            json.dump(unit_data, f)
    else:
        print(f"Unit available in file for {temp_unit_data[f'{name}']}")
        unit_data[f'{name}'] = {}
        unit_data[f'{name}']['label'] = temp_unit_data[f'{name}']['label']
        unit_data[f'{name}']['symbol'] = temp_unit_data[f'{name}']['symbol']
        unit_data[f'{name}']['id'] = temp_unit_data[f'{name}']['id']
        
        

In [ ]:
get_unit_id('unit_data.json', user2_data, unit_data, 'piece', 'u_piece', 'om2:one')
get_unit_id('unit_data.json', user2_data, unit_data, 'mass', 'kg', 'om2:kilogram')
get_unit_id('unit_data.json', user2_data, unit_data, 'volume', 'lt', 'om2:litre')
get_unit_id('unit_data.json', user1_data, unit_data, 'time', 'h', 'om2:hour')


In [ ]:
def get_resource_spec_id(file, user_data, res_spec_data, name, note, classification, default_unit_id):

    if name in res_spec_data:
        print(f"Specification {name} available")
        return

    # check we already have a unit file with an id
    if os.path.isfile(file):
        with open(file,'r') as f:
                temp_res_spec_data = json.loads(f.read())
    else:
        temp_res_spec_data = {}

    if name not in temp_res_spec_data:

        # Produce the query and variables vars to be signed
        variables = {
            "resourceSpecification": {
                "defaultUnitOfResource": default_unit_id,
                "name": name,
                "note": note,
                "resourceClassifiedAs": classification
            }
        }


        query = """mutation ($resourceSpecification:ResourceSpecificationCreateParams!){
                    createResourceSpecification(resourceSpecification:$resourceSpecification){
                        resourceSpecification{
                            id,
                            name
                        }
                    }
                }"""

        res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'])
#         print(res)

        # save the unit info
        res_spec_data[f'{name}'] = {}
        res_spec_data[f'{name}']['note'] = note
        res_spec_data[f'{name}']['classification'] = classification
        res_spec_data[f'{name}']['defaultUnit'] = default_unit_id
        res_spec_data[f'{name}']['id'] = res['data']['createResourceSpecification']['resourceSpecification']['id']

        # save data with id
        with open(file,'w') as f:
            json.dump(res_spec_data, f)
    else:
        print(f"Specification available in file for {temp_unit_data[f'{name}']}")
        res_spec_data[f'{name}'] = {}
        res_spec_data[f'{name}']['note'] = temp_res_spec_data[f'{name}']['note']
        res_spec_data[f'{name}']['classification'] = temp_res_spec_data[f'{name}']['classification']
        res_spec_data[f'{name}']['defaultUnit'] = temp_res_spec_data[f'{name}']['defaultUnit']
        res_spec_data[f'{name}']['id'] = temp_res_spec_data[f'{name}']['id']
        



In [ ]:
name = 'soap'
note = 'Specification for soap to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q34396'
default_unit_id = unit_data['mass']['id']
get_resource_spec_id('res_spec_data.json', user2_data, res_spec_data, name, note, classification, default_unit_id)

name = 'water'
note = 'Specification for water to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q283'
default_unit_id = unit_data['volume']['id']
get_resource_spec_id('res_spec_data.json', user2_data, res_spec_data, name, note, classification, default_unit_id)

name = 'cotton'
note = 'Specification for cotton to be used to sew the gowns'
classification = 'https://www.wikidata.org/wiki/Q11457'
default_unit_id = unit_data['mass']['id']
get_resource_spec_id('res_spec_data.json', user2_data, res_spec_data, name, note, classification, default_unit_id)

name = 'gown'
note = 'Specification for gowns'
classification = 'https://www.wikidata.org/wiki/Q89990310'
default_unit_id = unit_data['piece']['id']
get_resource_spec_id('res_spec_data.json', user2_data, res_spec_data, name, note, classification, default_unit_id)

name = 'surgical_operation'
note = 'Specification for surgical operations'
classification = 'https://www.wikidata.org/wiki/Q600236'
default_unit_id = unit_data['time']['id']
get_resource_spec_id('res_spec_data.json', user2_data, res_spec_data, name, note, classification, default_unit_id)

In [ ]:
from datetime import datetime, timezone

def create_resource(user_data, res_data, amount):
    
    provider = user_data['id']
    receiver = user_data['id']
    # Get the unit from the spec, no need to pass it     
    unit_id = [specs['defaultUnit'] for name, specs in res_spec_data.items() if specs['id'] == res_data['spec_id']][0]
    
    # Produce the query and variables vars to be signed
    # Getting the current date and time
    ts = datetime.now(timezone.utc).isoformat()

    variables = {
        "event": {
            "note": "update event",
            "action": "raise",
            "provider": provider, 
            "receiver": receiver,
            "hasPointInTime" : ts,
            "resourceQuantity": {
              "hasUnit": unit_id,
              "hasNumericalValue": amount 
            },
            "resourceConformsTo": res_data['spec_id']
        },
        "newInventoriedResource": { 
            "name": res_data['res_name'],
            "trackingIdentifier": res_data['res_ref_id']
        }
    }

    query = """mutation($event:EconomicEventCreateParams!, $newInventoriedResource:EconomicResourceCreateParams) {
                createEconomicEvent(event:$event, newInventoriedResource:$newInventoriedResource) {
                    economicEvent {
                        id
                        provider {
                            id
                            name
                            note
                        }
                        resourceQuantity {
                            hasNumericalValue
                            hasUnit {
                                label
                                symbol
                            }
                        }
                        resourceInventoriedAs {
                            id
                            trackingIdentifier  
                      }
                    }
                }
            }"""

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'])
#     print(res)

    # save the unit info
    res_data['id'] = res['data']['createEconomicEvent']['economicEvent']['resourceInventoriedAs']['id']
    
    return res['data']['createEconomicEvent']['economicEvent']['id'], ts




In [ ]:
import random

def get_resource(res_data, res_name, user_data, event_seq):
    
    res_data[f'{res_name}_res'] = {}
    cur_res = res_data[f'{res_name}_res']

    rnd = random.randint(0, 10000)
    cur_res['res_ref_id'] = f'{res_name}-{rnd}'
    cur_res['res_name'] = res_name
    cur_res['spec_id'] = res_spec_data[f'{res_name}']['id']


    event_id, ts = create_resource(user_data, cur_res, amount)

    event_seq.append({'ts': ts, 'event_id':event_id, 'action' : 'raise', 'res_name': cur_res['res_name'], 'res': cur_res['id']})

In [ ]:
# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []

In [ ]:
# We create the resources that will not be saved to file as it is assumed they are recreated at each run
# to be verified

res_name = 'soap'
amount = 100

get_resource(res_data, res_name, user2_data, event_seq)

In [ ]:
res_name = 'water'
amount = 50

get_resource(res_data, res_name, user2_data, event_seq)

In [ ]:
res_name = 'cotton'
amount = 20

get_resource(res_data, res_name, user2_data, event_seq)

In [ ]:
res_data

In [ ]:
event_seq

In [ ]:
def create_process(cur_process, user_data):

    # Produce the query and variables vars to be signed
    # Getting the current date and time
    ts = datetime.now(timezone.utc).isoformat()

    variables = {
      "process": {
        "name": cur_process['name'],
        "note": cur_process['note'],
        "hasBeginning": ts,
        "hasEnd": ts
      }
    }

    query = """mutation($process:ProcessCreateParams!) {
        createProcess(process: $process) {
            process {
                id
            }
        }
    }"""

    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'])
#     print(res)

    # save the unit info
    cur_process['id'] = res['data']['createProcess']['process']['id']

In [ ]:
def get_process(process_name, note, user_data):

    name = process_name.replace(' ', '_')

    process_data[f'{name}'] = {}

    cur_process = process_data[f'{name}']

    cur_process['name'] = process_name
    cur_process['note'] = note
    

    create_process(cur_process, user_data)    

In [ ]:
# create data structure to hold processes
process_data = {}

In [ ]:
process_name = 'Sew gown process'
note = f"Sew gown process performed by {user2_data['name']}"

get_process(process_name, note, user2_data)

In [ ]:
process_name = 'Use gown process'
note = f"Use gown process performed by {user1_data['name']}"

get_process(process_name, note, user1_data)

In [ ]:
process_name = 'Clean gown process'
note = f"Clean gown process performed by {user2_data['name']}"

get_process(process_name, note, user1_data)

In [ ]:
process_data

In [ ]:
# Function implements actions != transfer actions
def create_event(user_data, action, note, amount, process, \
                 existing_res=None, new_res=None, work_spec=None):

    if not action in ['accept', 'consume', 'modify', 'produce', 'work']:
        print(f"We do not support {action} yet")
        assert 1 == 2

    ts = datetime.now(timezone.utc).isoformat()
    variables = {
        "event": {
            "action": action,
            "note": note,
            "provider": user_data['id'],
            "receiver": user_data['id'],
            "hasPointInTime" : ts
        }
    }


    if action == 'work':
        # If action is work then the quantity is about the action
        variables['event']['effortQuantity'] = {}
        var_obj = variables['event']['effortQuantity']
        var_obj['hasUnit'] = work_spec['unit_id']
    else:
        # If action is not work then the quantity is about the resource
        variables['event']['resourceQuantity'] = {}
        var_obj = variables['event']['resourceQuantity']
        # find the unit from the resource's specification
        if action in ['produce']:
            _res = new_res
        else:
            _res = existing_res
        var_obj['hasUnit'] = [specs['defaultUnit'] for name, specs in res_spec_data.items() \
                              if specs['id'] == _res['spec_id']][0]

    
    var_obj['hasNumericalValue'] = amount


    if action in ['accept', 'consume', 'work']:
        # These actions are input to a process
        variables['event']['inputOf'] = process['id']
    elif action in ['modify', 'produce']:
        # These actions are output of a process
        variables['event']['outputOf'] = process['id']
        
    if action in ['accept', 'consume', 'modify']:
        # These actions require a resource id to act upon
        variables['event']['resourceInventoriedAs'] = existing_res['id']
        
    if action in ['work']:
        # Need to provide the specification of the type of work
        variables['event']['resourceConformsTo'] = work_spec['spec_id']
        
    if action in ['produce']:
        variables['newInventoriedResource'] = {};
        variables['newInventoriedResource']['name'] = new_res['res_name']
        variables['newInventoriedResource']['trackingIdentifier'] = new_res['res_ref_id']
        variables['event']['resourceConformsTo'] = new_res['spec_id']
        

    
    # Define the fields for the GraphQL response
    resource_quantity = """
                            hasNumericalValue
                            hasUnit {
                                label
                                symbol
                            }

                        """

    resource_body = f"""
                            id
                            name
                            onhandQuantity {{
                                {resource_quantity}
                            }}
                            accountingQuantity {{
                                {resource_quantity}
                            }}
                            primaryAccountable {{
                            id
                          }}
                    """

    response = f"""economicEvent {{
                        id
                        provider {{
                            id
                            name
                            note
                        }}
                        resourceQuantity {{
                            hasNumericalValue
                            hasUnit {{
                                label
                                symbol
                            }}
                        }}
                        toResourceInventoriedAs {{
                            {resource_body}
                        }}
                        resourceInventoriedAs {{
                            {resource_body}
                        }}
                    }}"""
                    
    if action in ['produce']:
        query = f"""
        mutation($event:EconomicEventCreateParams!, $newInventoriedResource: EconomicResourceCreateParams) {{
                createEconomicEvent(event:$event, newInventoriedResource:$newInventoriedResource) {{
                    {response}
                }}
            }}"""
    else:
        query = f"""
        mutation($event:EconomicEventCreateParams!) {{
                createEconomicEvent(event:$event) {{
                    {response}
                }}
            }}"""

#     print(query)
#     assert False
    res = send_signed(query, variables, user_data['username'], user_data['keyring']['eddsa'])
#     print(res)

    if action in ['produce']:
        # save the id of the new resource
        new_res['id'] = res['data']['createEconomicEvent']['economicEvent']['resourceInventoriedAs']['id']

    return res['data']['createEconomicEvent']['economicEvent']['id'], ts


In [ ]:
action = 'consume'
event_note='consume cotton for sewing'
amount = 10
cur_pros = process_data['Sew_gown_process']
cur_res = res_data['cotton_res']


event_id, ts = create_event(user2_data, action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['res_name'], 'res': cur_res['id']})
    


In [ ]:
action = 'produce'
event_note='produce gown'
amount = 1
cur_pros = process_data['Sew_gown_process']

res_data['gown_res'] = {
    "res_ref_id": f'gown-{random.randint(0, 10000)}',
    "res_name": 'gown',
    "spec_id": res_spec_data['gown']['id']
}
cur_res = res_data['gown_res']


event_id, ts = create_event(user2_data, action, event_note, amount=amount, process=cur_pros, \
                 new_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['res_name'], 'res': cur_res['id']})


In [ ]:
cur_res

In [ ]:
res_data

In [ ]:
res_spec_data

In [ ]:
process_data